In [1]:
import numpy as np
import cv2
import glob
import imutils
import scipy as cp
from imutils import perspective

In [2]:
def get_color_masks(img,window=20):
    
    #white
    lower_white = np.array([0,0,255-window], dtype=np.uint8)
    upper_white = np.array([255,window,255], dtype=np.uint8)
    white = cv2.inRange(img, lower_white, upper_white)
    
    #green
    lower_green = np.array([34, 50, 50],np.uint8)
    upper_green = np.array([72, 255, 255],np.uint8)
    green=cv2.inRange(img,lower_green,upper_green)
    
    #yellow
    lower_yellow = np.array([20, 50, 50],np.uint8)
    upper_yellow = np.array([33, 255, 255],np.uint8)
    yellow=cv2.inRange(img,lower_yellow,upper_yellow)
    
    #red
    #red has to be done twice because the revolving ends and starts by red
    lower_red1 = np.array([0,50,50])
    upper_red1 = np.array([9,255,255])
    lower_red2 = np.array([170,50,50])
    upper_red2 = np.array([180,255,255])
    mask0 = cv2.inRange(img, lower_red1, upper_red1)
    mask1 = cv2.inRange(img, lower_red2, upper_red2)
    red = mask0+mask1
    return white,red,yellow,green
    

In [3]:
def undistort_img():
    # Prepare object points 0,0,0 ... 8,5,0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    obj_pts = np.zeros((6*9,3), np.float32)
    obj_pts[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    # Stores all object points & img points from all images
    objpoints = []
    imgpoints = []
    # Get directory for all calibration images
    images = glob.glob('camera_cal/*.png')
    for indx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), flags=cv2.CALIB_CB_ADAPTIVE_THRESH)
        if ret == True:
            objpoints.append(obj_pts)
            corners2 = cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),criteria)
            imgpoints.append(corners2)
    # Test undistortion on img
    img_size = (img.shape[1], img.shape[0])
    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None,None)
    map1,map2=cv2.initUndistortRectifyMap(mtx,dist,None,mtx,(img.shape[1],img.shape[0]), cv2.CV_32FC1)
    
    return map1,map2

In [33]:

def get_warp_perspective(IMAGE_H,IMAGE_W,):
    src = np.float32([[0, IMAGE_H], [2500, IMAGE_H], [0, 0], [IMAGE_W, 0]])
    dst = np.float32([[300, IMAGE_H], [600, IMAGE_H], [0, 0], [IMAGE_W, 0]])
    M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
    #Minv = cv2.getPerspectiveTransform(dst, src) # Inverse transformation

     # Apply np slicing for ROI crop
    return  M

In [18]:
def distance_to_camera(knownWidth, focalLength, perWidth):
    # compute and return the distance from the maker to the camera
    return (knownWidth * focalLength) / perWidth

In [19]:
def find_marker(image):
    # convert the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)
 
    # find the contours in the edged image and keep the largest one;
    # we'll assume that this is our piece of paper in the image
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key = cv2.contourArea)
 
    # compute the bounding box of the of the paper region and return it
    return cv2.minAreaRect(c)
# initialize the known distance from the camera to the object, which
# in this case is 24 inches
KNOWN_DISTANCE = 24.0
 
# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 11.0
 
# load the furst image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize
# the focal length
image = cv2.imread("stoptest.jpg")
marker = find_marker(image)
focalLength = (marker[1][0] * KNOWN_DISTANCE) / KNOWN_WIDTH

In [20]:
def detect_stop_distance(img):

    knownWitdh=12 #to be measured
    focalLength=5 #to be measured
    contours= cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    c = max(contours, key = cv2.contourArea)

    peri = cv2.arcLength(c, True)

    approx = cv2.approxPolyDP(c, 0.015 * peri, True)
    
    if len(approx) == 8:
        perWidth = cv2.minAreaRect(approx)
        distance = (knownWidth * focalLength) / perWidth[1][0]
        return 1,distance
    else:
        return 0,0

In [57]:

img2=cv2.imread('route.png')
IMAGE_H = 480
IMAGE_W = 640
map1,map2=undistort_img()
M = get_warp_perspective(IMAGE_H,IMAGE_W)





distort=cv2.remap(img2,map1,map2,interpolation=cv2.INTER_LINEAR)

distort = distort[75:(IMAGE_H-75), 0:IMAGE_W]
e1=cv2.getTickCount()
warped = cv2.warpPerspective(distort, M, (IMAGE_W, IMAGE_H),flags=cv2.INTER_NEAREST) 

e2=cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()
print(time)
cv2.imshow('frame',warped)
cv2.waitKey(0) 
cv2.destroyAllWindows()

0.0024686


In [85]:
img2=cv2.imread('route.png')
IMAGE_H = 480
IMAGE_W = 640
map1,map2=undistort_img()


distort=cv2.remap(img2,map1,map2,interpolation=cv2.INTER_LINEAR)
#distort = imutils.translate(distort, 175, 0)
img_hsv=cv2.cvtColor(distort,cv2.COLOR_BGR2HSV)



window=20
lower_white = np.array([0,0,255-window], dtype=np.uint8)
upper_white = np.array([255,window,255], dtype=np.uint8)

cxs=[]
cys=[]
e1=cv2.getTickCount()
#e1=cv2.getTickCount()
for i in range (350,100,-100):
    slice = img_hsv[i:i+100, 0:640]
    
    mask = cv2.inRange(slice, lower_white, upper_white)
    contours=cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    contours= imutils.grab_contours(contours)
    c = max(contours, key = cv2.contourArea)
    peri = cv2.arcLength(c, True)
    c = cv2.approxPolyDP(c, 0.02 * peri, True)
    extLeft = c[c[:, :, 0].argmin()][0]
    extRight = c[c[:, :, 0].argmax()][0]
    diff=extRight[0]-extLeft[0]
    if (diff<200):

        #rect = cv2.minAreaRect(c)
   
        #box = cv2.boxPoints(rect)
        #box = np.int0(box)
        #box[:,1]=box[:,1]+i
        #cv2.drawContours(img2,[box],0,(0,0,255),2)

        M = cv2.moments(c)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cxs.append(cx)
        cys.append(cy+i)
        #cv2.circle(img2, (cx, cy+i), 4, (255, 0, 255), 2)
    else:
        for j in range (0,diff,80):
            sub_slice = slice[0:100, extLeft[0]+j:extLeft[0]+j+100]
            mask = cv2.inRange(sub_slice, lower_white, upper_white)
            contours=cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
            contours= imutils.grab_contours(contours)
            c = max(contours, key = cv2.contourArea)
            peri = cv2.arcLength(c, True)
            c = cv2.approxPolyDP(c, 0.02 * peri, True)
            #rect = cv2.minAreaRect(c)
   
            #box = cv2.boxPoints(rect)
            #box = np.int0(box)
            #box[:,1]=box[:,1]+i
            #box[:,0]=box[:,0]+j+extLeft[0]
            #cv2.drawContours(img2,[box],0,(0,0,255),2)

            M = cv2.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cxs.append(cx+j+extLeft[0])
            cys.append(cy+i)
            #cv2.circle(img2, (cx+j+extLeft[0], cy+i), 4, (255, 0, 255), 2)
        
        

for i in range (0,len(cxs)-1):
    cv2.arrowedLine(img2, (cxs[i],cys[i]), (cxs[i+1],cys[i+1]), (0,0,255), 2)
    ret = int(round(cv2.fastAtan2((cys[i]-cys[i+1]),(cxs[i+1]-cxs[i]))))
    #print(ret)
cxs = np.diff(cxs)
cys = -1*np.diff(cys)
dist=np.hypot(cxs,cys)
e2=cv2.getTickCount()
time=(e2-e1)/cv2.getTickFrequency()
cv2.imshow('frame',img2)
cv2.waitKey(0) 
cv2.destroyAllWindows()
print(time)
#size = np.size(test)
#skel = np.zeros(test.shape,np.uint8)
#test2=imutils.skeletonize(test,(3,3))
#test2 = cv2.morphologyEx(test2, cv2.MORPH_OPEN, (3,3))

#z = np.polyfit(x,y,5)
#f = np.poly1d(z)
#ploty = np.linspace(x_min, x_max, np.int((x_max-x_min)/10) )
#new=f(ploty)
#left = np.array([np.transpose(np.vstack([new,ploty]))])
#cv2.polylines(warped, np.int_(left),False, (45,25,0),2)
#time = (e2 - e1)/ cv2.getTickFrequency()
#print(time)
#skel=cv2.cvtColor(skel,cv2.COLOR_GRAY2BGR)
#skeletonize=cv2.bitwise_or(warped,skel)


0.002641


In [13]:
ret = int(round(cv2.fastAtan2(50,50)))
print(ret)
surf=cv2.SURF(400)

45


AttributeError: module 'cv2.cv2' has no attribute 'SURF'

In [ ]:
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
done = False
e1=cv2.getTickCount()
while( not done):
    eroded = cv2.erode(test,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(test,temp)
    skel = cv2.bitwise_or(skel,temp)
    test = eroded.copy()

    zeros = size - cv2.countNonZero(test)
    if zeros==size:
        done = True
e2=cv2.getTickCount()

In [ ]:
#y=c[:,0][:,0]
#x=c[:,0][:,1]
#x_min=x[np.argmin(x)]
#x_max=x[np.argmax(x)]
#z = np.polyfit(x,y,5)
#f = np.poly1d(z)
#ploty = np.linspace(x_min, x_max, np.int((x_max-x_min)/10) )
#new=f(ploty)
#left = np.array([np.transpose(np.vstack([new,ploty]))])
#cv2.polylines(warped, np.int_(left),False, (45,25,0),2)

#size = np.size(test)
#skel = np.zeros(test.shape,np.uint8)
#test2=imutils.skeletonize(test,(3,3))
#test2 = cv2.morphologyEx(test2, cv2.MORPH_OPEN, (3,3))

#z = np.polyfit(x,y,5)
#f = np.poly1d(z)
#ploty = np.linspace(x_min, x_max, np.int((x_max-x_min)/10) )
#new=f(ploty)
#left = np.array([np.transpose(np.vstack([new,ploty]))])
#cv2.polylines(warped, np.int_(left),False, (45,25,0),2)
#time = (e2 - e1)/ cv2.getTickFrequency()
#print(time)
#skel=cv2.cvtColor(skel,cv2.COLOR_GRAY2BGR)
#skeletonize=cv2.bitwise_or(warped,skel)